A desktop app by Mike Bourne to review and update sold prices of properties in a given area

<b>GUI</b><br>
Creating using tkinter

Display output: 
House Number, Date Sold, Price

User can: 
View all, records, Search an entry, Add entry, Update entry, Delete, Close app.

<b>DATABASE</b><br>

 Internal database using sqlite
 
 
 ---------------------------------------------------

<h4>Create the SQL backend database</h4>

In [1]:
import sqlite3 

class Database:
    
    # constructor  - includes creating new table 'hptable' if not already present
    
    def __init__(self,db):
        self.conn=sqlite3.connect(db)
        self.cur=self.conn.cursor()
        self.cur.execute("CREATE TABLE IF NOT EXISTS hptable (hp_id INTEGER PRIMARY KEY,number TEXT,price INTEGER,date TEXT)")
        self.conn.commit()
        
    # inset into hptable
    
    def insert(self,num,hp,date):
        self.cur.execute("INSERT INTO hptable VALUES(NULL,?,?,?)",(num,hp,date))
        self.conn.commit()

    # view into hptable

    def viewtable(self):
        self.cur.execute("SELECT * FROM hptable")
        rows=self.cur.fetchall()
        return rows

    # search into hptable

    def search(self,house_num="",price="",date=""):
        self.cur.execute("SELECT * FROM hptable WHERE number =? OR price =? OR date =?", (house_num,price,date))
        rows=self.cur.fetchall()
        return rows

    # delete into hptable

    def delete(self,idin):
        self.cur.execute("DELETE FROM hptable WHERE hp_id =?", (idin,))
        self.conn.commit()

    # update the hptable table

    def update(self,idin,house_num,price,date):
        self.cur.execute("UPDATE hptable SET number=?,price=?,date=? WHERE hp_id =?", (house_num,price,date,idin))
        self.conn.commit()

    # close the hptable table when window is destroyed

    def __del__(self):
        self.conn.close()      
        


<h4>Create the front-end GUI:</h4>

In [16]:
from tkinter import *
window=Tk()
window.title("Mike's House Price Database")

# create an database object from class Database 
# use the sql database 'houseprice.db'

database=Database("houseprice.db")

# command functions defined

def view_command():
    list1.delete(0,END)
    for row in database.viewtable():
        list1.insert(END,row)
        
def search_command():
    list1.delete(0,END)
    for row in database.search(house_no_text.get(),price_text.get(),date_text.get()):
        list1.insert(END,row)

def add_command():
    database.insert(house_no_text.get(),price_text.get(),date_text.get().title())

def update_command():
    database.update(selected_index[0],house_no_text.get(),price_text.get(),date_text.get().title())
    search_command()

def delete_command():
    database.delete(selected_index[0])
    search_command()
    
    
# get the row selected in the list
    
def get_selected_row(event):
    global selected_index
    try:
        index=list1.curselection()[0]
        selected_index = list1.get(index)
    except:
        pass
    
    # display this selected entry in the text boxes

    e1.delete(0,END)
    e1.insert(END,selected_index[1])
    e2.delete(0,END)
    e2.insert(END,selected_index[2])
    e3.delete(0,END)
    e3.insert(END,selected_index[3])
    
    
#####################
# Create the GUI
#####################

# Create static text boxes

l1=Label(window,text="House No")
l1.grid(row=0,column=0)

l2=Label(window,text="Date Sold")
l2.grid(row=0,column=2)

l2=Label(window,text="Price")
l2.grid(row=1,column=0)


# Create text boxes to display database output and which can be edited

house_no_text=StringVar()
e1=Entry(window,textvariable=house_no_text)
e1.grid(row=0,column=1)

date_text=StringVar()
e2=Entry(window,textvariable=date_text,width=20)
e2.grid(row=0,column=3)

price_text=StringVar()
e3=Entry(window,textvariable=price_text)
e3.grid(row=1,column=1)

# Create text boxes to display entries when search
# Including a scrollbar

list1=Listbox(window,height=6,width=32)
list1.grid(row=2,column=0, columnspan=2, rowspan=6)

# define bind arguement for the ListBox

list1.bind('<<ListboxSelect>>',get_selected_row)

myscrollbar=Scrollbar(window,orient="vertical",command=list1.yview)
myscrollbar.grid(row=2,column=2,rowspan=6)
list1.configure(yscrollcommand=myscrollbar.set)

# Create buttons which will be used to interact with the sql database

b1=Button(window,text="View All", width=12, command=view_command)
b1.grid(row=1, column=3)

b2=Button(window,text="Search Entry", width=12, command=search_command)
b2.grid(row=2, column=3)

b3=Button(window,text="Add Entry", width=12, command=add_command)
b3.grid(row=3, column=3)

b4=Button(window,text="Update", width=12, command=update_command)
b4.grid(row=4, column=3)

b5=Button(window,text="Delete", width=12, command=delete_command)
b5.grid(row=5, column=3)

b6=Button(window,text="Close", width=12, command=window.destroy)
b6.grid(row=6, column=3)

# Open the application

window.mainloop()
